# Python Libraries


In [ ]:
import os
import itertools

import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB3
from keras.applications import VGG16

# ignore the warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def loading_the_data(data_dir):
    filepaths = []
    labels = []

    folds = os.listdir(data_dir)

    for fold in folds:
        foldpath = os.path.join(data_dir, fold)
        filelist = os.listdir(foldpath)
        for file in filelist:
            fpath = os.path.join(foldpath, file)
            
            filepaths.append(fpath)
            labels.append(fold)

    Fseries = pd.Series(filepaths, name='filepaths')
    Lseries = pd.Series(labels, name='labels')

    df = pd.concat([Fseries, Lseries], axis=1)
    
    return df

In [ ]:
data_dir = '/kaggle/input/tomatoleaf/tomato/train'
df = loading_the_data(data_dir)

df

In [ ]:
data_balance = df.labels.value_counts()
data_balance

In [ ]:
def custom_autopct(pct):
    total = sum(data_balance)
    val = int(round(pct*total/100.0))
    return "{:.1f}%\n({:d})".format(pct, val)


In [ ]:
plt.pie(data_balance, labels = data_balance.index, autopct=custom_autopct, colors = ["#FF0000", "#FF69B4", "#0000FF", "#FFFFFF", "#00FF00", "#800080", "#FFFF00", "#A52A2A", "#40E0D0", "#FFA500"])
plt.title("Data balance")
plt.axis("equal")
plt.show()

In [ ]:
def model_performance(history, Epochs):
    tr_acc = history.history['accuracy']
    tr_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    
    Epochs = [i+1 for i in range(len(tr_acc))]
    
    plt.figure(figsize= (20, 8))
    plt.style.use('fivethirtyeight')
    
    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
    plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
    plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout
    plt.show()

In [ ]:
def model_evaluation(model):
    train_score = model.evaluate(train_gen, verbose= 1)
    valid_score = model.evaluate(valid_gen, verbose= 1)
    test_score = model.evaluate(test_gen, verbose= 1)
    
    print("Train Loss: ", train_score[0])
    print("Train Accuracy: ", train_score[1])
    print('-' * 20)
    print("Validation Loss: ", valid_score[0])
    print("Validation Accuracy: ", valid_score[1])
    print('-' * 20)
    print("Test Loss: ", test_score[0])
    print("Test Accuracy: ", test_score[1])

In [ ]:
def get_pred(model, test_gen):
    
    preds = model.predict(test_gen)
    y_pred = np.argmax(preds, axis = 1)
    
    return y_pred


In [ ]:
def plot_confusion_matrix(test_gen, y_pred):
    
    g_dict = test_gen.class_indices
    classes = list(g_dict.keys())
    
    cm = confusion_matrix(test_gen.classes, y_pred)

    plt.figure(figsize= (10, 10))
    plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation= 45, fontsize=8)  
    plt.yticks(tick_marks, classes)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')
    
    
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_model(model, test_gen):
    y_true = test_gen.classes  

    
    y_pred_probs = model.predict(test_gen)  

    # Convert probabilities to class labels
    y_pred = np.argmax(y_pred_probs, axis=1)  

    # Print classification report
    print("Classification Report:\n")
    print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    class_labels = list(test_gen.class_indices.keys())

    # Print confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    # Compute True Positives (TP) and False Positives (FP) per class
    TP = np.diag(cm)  # Diagonal elements are True Positives
    FP = np.sum(cm, axis=0) - TP  # Column-wise sum minus TP

    # Print TP and FP counts
    for i, label in enumerate(class_labels):
        print(f"{label}: True Positives = {TP[i]}, False Positives = {FP[i]}")


In [ ]:
def conv_block(filters, act='relu'):
    
    block = Sequential()
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(BatchNormalization())
    block.add(MaxPooling2D())
    
    return block

In [ ]:
def dense_block(units, dropout_rate, act='relu'):
    
    block = Sequential()
    block.add(Dense(units, activation=act))
    block.add(BatchNormalization())
    block.add(Dropout(dropout_rate))
    
    return block

In [ ]:
from sklearn.model_selection import train_test_split

#85 5 10
train_df, temp_df = train_test_split(df, train_size=0.80, shuffle=True, random_state=42)
valid_df, test_df = train_test_split(temp_df, train_size=1/2, shuffle=True, random_state=42)

# #70 20 10
# train_df, temp_df = train_test_split(df, train_size=0.7, shuffle=True, random_state=42)
# valid_df, test_df = train_test_split(temp_df, train_size=1/3, shuffle=True, random_state=42)



In [ ]:
batch_size = 16
img_size = (224, 224)

tr_gen = ImageDataGenerator(rescale=1. / 255,
                           rotation_range=30,      
                           width_shift_range=0.2,  
                           height_shift_range=0.2, 
                           shear_range=0.2,        
                           zoom_range=0.2,         
                           horizontal_flip=True,   
                           fill_mode="nearest",    
                           validation_split=0.2)



ts_gen = ImageDataGenerator(rescale=1. / 255,
                           rotation_range=30,      
                           width_shift_range=0.2,  
                           height_shift_range=0.2, 
                           shear_range=0.2,        
                           zoom_range=0.2,         
                           horizontal_flip=True,   
                           fill_mode="nearest",    
                           validation_split=0.2)


train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= False, batch_size= batch_size)

In [ ]:
g_dict = train_gen.class_indices     
classes = list(g_dict.keys())       
images, labels = next(train_gen)      

# ploting the patch size samples
plt.figure(figsize= (20, 20))

for i in range(batch_size):
    plt.subplot(4, 4, i + 1)
    image = images[i]
    plt.imshow(image)
    index = np.argmax(labels[i])  # get image index
    class_name = classes[index]   # get class of image
    plt.title(class_name, color= 'black', fontsize= 16)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

class_counts = len(list(train_gen.class_indices.keys()))

# CNN Model Building

In [ ]:
cnn_model = Sequential()

cnn_model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu", input_shape= img_shape))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D())

cnn_model.add(conv_block(32))

cnn_model.add(conv_block(64))

cnn_model.add(conv_block(128))

cnn_model.add(conv_block(256))

cnn_model.add(conv_block(512))

cnn_model.add(Flatten())

cnn_model.add(dense_block(256, 0.5))

cnn_model.add(dense_block(128, 0.3))

cnn_model.add(dense_block(64, 0.2))

cnn_model.add(dense_block(32, 0.2))

cnn_model.add(Dense(class_counts, activation = "softmax"))

In [ ]:
cnn_model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

cnn_model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model


plot_model(cnn_model, to_file="model_structure.png", show_shapes=True, show_layer_names=True)

import matplotlib.pyplot as plt
import cv2


img = cv2.imread("model_structure.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis("off") 
plt.show()


In [ ]:
epochs = 10   

history = cnn_model.fit(train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, shuffle= False)

# CNN Model Performance - Prediction

In [ ]:
model_performance(history, epochs)

In [ ]:
model_evaluation(cnn_model)

In [ ]:
y_pred = get_pred(cnn_model, test_gen)

plot_confusion_matrix(test_gen, y_pred)

# EfficientNetB3 Model Building

In [ ]:

base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape = img_shape, pooling= None)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = dense_block(128, 0.5)(x)
x = dense_block(32, 0.2)(x)
predictions = Dense(class_counts, activation = "softmax")(x)    # output layer with softmax activation


EfficientNetB3_model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
from tensorflow.keras.optimizers import AdamW
EfficientNetB3_model.compile(optimizer=AdamW(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
epochs = 30   

# EfficientNetB3_history = EfficientNetB3_model.fit(train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, shuffle= False)

In [ ]:
import tensorflow as tf
import time
import numpy as np
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import precision_score, recall_score, f1_score
from tabulate import tabulate


print(f"\n🔹 Training EfficientNetB3 with Adam optimizer...\n")
start_time = time.time()
EfficientNetB3_history = EfficientNetB3_model.fit(train_gen, epochs=epochs, verbose=1, validation_data=valid_gen, shuffle=False)
end_time = time.time()


model_performance(EfficientNetB3_history, epochs)


test_loss, test_acc = EfficientNetB3_model.evaluate(test_gen, verbose=0)


y_pred = get_pred(EfficientNetB3_model, test_gen)
y_true = test_gen.classes  


precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")
val_acc = max(EfficientNetB3_history.history["val_accuracy"])  


plot_confusion_matrix(test_gen, y_pred)


evaluate_model(EfficientNetB3_model, test_gen)


results = [["Adam", test_acc, test_loss, precision, recall, f1, end_time - start_time, val_acc]]

headers = ["Optimizer", "Accuracy", "Loss", "Precision", "Recall", "F1-Score", "Training Time (s)", "Validation Acc"]
print("\n🔹 Model Performance Summary:\n")
print(tabulate(results, headers=headers, tablefmt="pretty"))

In [ ]:
from tensorflow.keras.utils import plot_model



num_layers_to_show = int(len(EfficientNetB3_model.layers) * 0.1)  


sub_model = tf.keras.Model(inputs=EfficientNetB3_model.input, 
                           outputs=EfficientNetB3_model.layers[-num_layers_to_show].output)


plot_model(sub_model, to_file="sub_model.png", show_shapes=True, show_layer_names=True, dpi=150)


import matplotlib.pyplot as plt
import cv2


img = cv2.imread("sub_model.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis("off") 
plt.show()

# EfficientNetB3 Model Performance - Prediction

In [ ]:
model_performance(EfficientNetB3_history, epochs)

In [ ]:
model_evaluation(EfficientNetB3_model)

In [ ]:
y_pred = get_pred(EfficientNetB3_model, test_gen)


plot_confusion_matrix(test_gen, y_pred)

In [ ]:
evaluate_model(EfficientNetB3_model, test_gen)

In [ ]:
# Save the full model (architecture + weights)
EfficientNetB3_model.save("efficientnet_b3_model.h5")

# Save only the weights (optional, if you want to load separately)
EfficientNetB3_model.save_weights("efficientnet_b3.weights.h5")


# VGG16 Model Building

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = img_shape, pooling= 'max')

for layer in base_model.layers[:15]:
    layer.trainable = False
    
    

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.2)(x)   # # Dropout layer to prevent overfitting
x = Dense(256, activation = 'relu')(x)
x = Dense(128, activation = 'relu')(x)
x = Dense(32, activation = 'relu')(x)
predictions = Dense(class_counts, activation = "sigmoid")(x)    # output layer with softmax activation


VGG16_model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
for layer in VGG16_model.layers:
    print(layer.name, layer.trainable)

In [ ]:
VGG16_model.compile(optimizer=Adamax(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

VGG16_model.summary()

In [ ]:
epochs = 20   

VGG16_history = VGG16_model.fit(train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, shuffle= False)

# VGG16 Model Performance - Prediction

In [ ]:
model_performance(VGG16_history, epochs)

In [ ]:
model_evaluation(VGG16_model)

In [ ]:
evaluate_model(VGG16_model, test_gen)

In [ ]:
from tensorflow.keras.applications import ResNet50
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=img_shape, pooling='max')


for layer in resnet_model.layers[:10]:  
    layer.trainable = False


x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)  
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(32, activation='relu')(x)


predictions = Dense(class_counts, activation="softmax")(x)  


ResNet50_model = Model(inputs=resnet_model.input, outputs=predictions)


ResNet50_model.compile(optimizer=Adam(learning_rate=0.0001), 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])


In [ ]:
history = ResNet50_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=10
)

In [ ]:
evaluate_model(ResNet50_model, test_gen)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2


model = load_model('efficientnet_b3_model.h5')


class_labels = ["Tomato___Bacterial_spot", "Tomato___Early_blight", "Tomato___Late_blight",
                "Tomato___Leaf_Mold", "Tomato___Septoria_leaf_spot", "Tomato___Spider_mites",
                "Tomato___Target_Spot", "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
                "Tomato___Tomato_mosaic_virus", "Tomato___healthy"]

def preprocess_image(image_path):
    
    img = cv2.imread(image_path)  
    if img is None:
        print("Error")
        exit(1)  
    img = cv2.resize(img, (224, 224))  
    img = img / 255.0  
    img = np.expand_dims(img, axis=0)  
    return img


image_path = input("Enter the path of the image: ").strip()


image = preprocess_image(image_path)
predictions = model.predict(image)
predicted_class = np.argmax(predictions)  
predicted_label = class_labels[predicted_class]


print(f"Predicted Disease: {predicted_label}")
